In [31]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.image as mpimg
import matplotlib.animation as animation

import time
import sqlite3
import ipystata

from sklearn import manifold

import figures

IPyStata is loaded in batch mode.


# Trade Matrix Calculated From UN COMTRADE Dataset

The matrices calculated here are created from the publicly available data from UN Comtrade dataset (2001-2009) and Feenstra Dataset (1962-2000). I have collected data on the types of products that are traded between a sample of 70 countries between years 1962 and 2009.

These datasets use Standardized International Trade Codes (SITC), which are used for classify types of goods that are globally traded. SITC codes consist of 4 Digits. Each digit determines a more specific category from left to right. For example: 
* 1 - Beverages and Tobacco 
    * 11 - Beverages
        * 112 - Alcoholic beverages
            * 112.3 - Beer made from malt (including ale, stout, and porter)

Below is a list of the top level codes:
0. Food and live animals
1. Beverages and tobacco
2. Crude materials, inedible, except fuels
3. Mineral, fuels, lubricants and related materials
4. Animal and vegetable oils, fats and waxes
5. Chemicals and related products, n.e.s.
6. Manufactured goods classified chiefly by material
7. Machinery and transport equipment
8. Miscellaeous manufactured articles
9. Commodities and transactions not classified elsewhere

You can find more information regarding SITC Classifications registry at http://unstats.un.org/unsd/cr/registry/regcst.asp?Cl=14

Below is an excerpt from all the data I have concatenated together from these sources

In [3]:
con = sqlite3.connect('/Volumes/iMac HD/git/insight-demo/data/sitc.db')
products_trade = pd.read_sql('select year, importer, exporter, sitc4, value from "Feenstra-SITC"', con)

,year,importer,exporter,sitc4,value
0,2000,World,World,,5895564
1,2000,World,World,0011,1201050
2,2000,World,World,0011,518532
3,2000,World,World,0011,2179618
4,2000,World,World,0012,120856


In [29]:
products_trade[(products_trade.importer == "Turkey") 
               & (products_trade.exporter != "World")].sort_values(by=['year',
                                                                       'importer','exporter','sitc4'])

,year,importer,exporter,sitc4,value
1113510,1962,Turkey,Afghanistan,7720,6
1112951,1962,Turkey,Algeria,2924,2
1112952,1962,Turkey,Algeria,6623,12
1116277,1962,Turkey,Areas NES,6832,1
1113251,1962,Turkey,Argentina,2112,42
1113252,1962,Turkey,Argentina,2682,57
1113253,1962,Turkey,Argentina,4113,40
1113254,1962,Turkey,Argentina,5322,3
1113255,1962,Turkey,Argentina,5416,9
1116263,1962,Turkey,Australia,2111,129


In [4]:
products_trade.describe()

,year,importer,exporter,sitc4,value
count,27573764,27573764,27573764,27573764,27573764
unique,39,203,201,1690,375804
top,1997,World,World,5417,1
freq,882248,2410486,3199028,119737,1710732


Using this dataset I have created role equivalence measure for a sample of 70 countries from the world that makes up 95% of the World's GDP. This measure is used in the sociology and management literature and calculates the similarities between nodes in a network by considering the correlations of valued ties. 

To calculate this measure I have first calculated an export product category share vector (EPSV) and an import product category share vector (IPSV) using the formulas below:

## $EPSV_{it} = \frac{exports_{ikt}}{\sum exports_{ikt}}$
## $IPSV_{it} = \frac{imports_{ikt}}{\sum imports_{ikt}}$

I concatenated these two vectors to get a product share vector (PSV) and finally I created a trade matrix based on the correlations of product share vectors for each country.

### Role Equivalence in Trade:

## $Attribute * r (PSV_{it}, PSV_{jt})$

Below is the symmetric trade matrix for 1962 where each cell reports for the correlation on the type of products trade between each pair of countries:

In [19]:
tradeyear1962 = pd.read_csv('data/1962_rev.csv')
tradeyear1962

,year,FEENSTRAcode,value_stacked117100,value_stacked135040,value_stacked137880,value_stacked138180,value_stacked141400,value_stacked162880,value_stacked163840,value_stacked165660,...,value_stacked552460,value_stacked553520,value_stacked555780,value_stacked557520,value_stacked557560,value_stacked583480,value_stacked586160,value_stacked586420,value_stacked710360,value_stacked715540
0,1962,117100,1.000000,0.580727,0.298044,0.470812,0.444656,0.142892,0.154735,0.264662,...,0.179566,0.171661,0.258835,0.420187,0.136996,0.253699,0.140088,0.262556,0.714803,0.466513
1,1962,135040,0.580727,1.000000,0.444529,0.281030,0.090011,0.110292,0.183642,0.111618,...,0.061212,0.187686,0.156467,0.175202,0.036515,0.255572,0.149251,0.080086,0.190184,0.088327
2,1962,137880,0.298044,0.444529,1.000000,0.376566,0.109824,0.091489,0.107405,0.409892,...,0.045924,0.083095,0.168233,0.132455,0.057890,0.204983,0.164935,0.498706,0.111423,0.062776
3,1962,138180,0.470812,0.281030,0.376566,1.000000,0.552737,0.022472,0.062017,0.169648,...,0.013675,0.034875,0.085897,0.010670,0.081922,0.194281,0.239873,0.210107,0.719614,0.534223
4,1962,141400,0.444656,0.090011,0.109824,0.552737,1.000000,0.649721,0.636258,0.492181,...,0.096211,0.050254,0.042241,0.101738,0.103024,0.057377,0.007624,0.059112,0.629279,0.507022
5,1962,162880,0.142892,0.110292,0.091489,0.022472,0.649721,1.000000,0.958899,0.502106,...,0.155786,0.027763,0.031853,0.152697,0.043021,0.059916,0.044805,0.117732,0.056770,0.052964
6,1962,163840,0.154735,0.183642,0.107405,0.062017,0.636258,0.958899,1.000000,0.513306,...,0.266134,0.038619,0.035831,0.152180,0.060850,0.110427,0.092189,0.204426,0.078699,0.073516
7,1962,165660,0.264662,0.111618,0.409892,0.169648,0.492181,0.502106,0.513306,1.000000,...,0.147793,0.070416,0.110475,0.151310,0.071882,0.112146,0.021339,0.287527,0.197014,0.168632
8,1962,211240,0.526961,0.238687,0.256704,0.062096,0.095077,0.128596,0.155802,0.224008,...,0.649362,0.128504,0.598085,0.666478,0.176563,0.098815,0.107257,0.484297,0.281046,0.122748
9,1962,218400,0.261812,0.137312,0.152630,0.103984,0.123988,0.016184,0.027102,0.150207,...,0.056082,0.045907,0.156476,0.476716,0.341011,0.125065,0.029067,0.203066,0.265478,0.123454


In [13]:
mat = np.array(tradeyear1962.ix[:, 2:])
mat

array([[ 1.       ,  0.5807274,  0.2980445, ...,  0.2625561,  0.7148025,
         0.4665128],
       [ 0.5807274,  1.       ,  0.4445291, ...,  0.0800861,  0.1901837,
         0.0883269],
       [ 0.2980445,  0.4445291,  1.       , ...,  0.4987055,  0.111423 ,
         0.0627757],
       ..., 
       [ 0.2625561,  0.0800861,  0.4987055, ...,  1.       ,  0.2308287,
         0.1192481],
       [ 0.7148025,  0.1901837,  0.111423 , ...,  0.2308287,  1.       ,
         0.8239651],
       [ 0.4665128,  0.0883269,  0.0627757, ...,  0.1192481,  0.8239651,
         1.       ]])

Below is a dataset from other variables I have collected in my research regarding crises of sovereignty. In this case I have observations regarding the number of revolutionary situations experienced in a country reported by New York Times. This data comes from Banks' Cross National Time Series Dataset. 

In [14]:
attributevectors1962 = pd.read_csv('data/1962_rev_vector.csv')
attributevectors1962

,FEENSTRAcode,FEENSTRAcountry,revolutions,rev1
0,330320,Argentina,5,1
1,710360,Australia,0,0
2,550400,Austria,0,0
3,583480,Hungary,0,0
4,530560,Belgium-Lux,0,0
5,330680,Bolivia,0,0
6,330760,Brazil,0,0
7,451040,Myanmar,1,1
8,211240,Canada,0,0
9,141400,Cent.Afr.Rep,0,0


In [20]:
rev = np.array(attributevectors1962.ix[:, -1:])
countries = np.array(attributevectors1962.ix[:, 1:2])

## Calculating Role Equivalence in Trade for Revolutionary Situations For the Year 1962

In [17]:
np.dot(mat, rev)

array([[ 3.8099826],
       [ 2.3652151],
       [ 1.9682156],
       [ 2.8761957],
       [ 3.0338606],
       [ 1.827923 ],
       [ 2.1817397],
       [ 2.5984015],
       [ 2.5734357],
       [ 2.4441077],
       [ 2.5358198],
       [ 1.5233188],
       [ 2.2375083],
       [ 1.2419277],
       [ 1.632814 ],
       [ 2.4545673],
       [ 3.3600457],
       [ 3.4648224],
       [ 2.6653814],
       [ 3.1787596],
       [ 0.8942851],
       [ 2.0259876],
       [ 2.0492814],
       [ 2.5029612],
       [ 3.0113671],
       [ 3.3443743],
       [ 1.5162731],
       [ 2.4829132],
       [ 1.5366884],
       [ 3.2397006],
       [ 1.5210828],
       [ 1.5959696],
       [ 2.1313965],
       [ 1.6686855],
       [ 1.5266527],
       [ 1.0509686],
       [ 2.4603351],
       [ 2.0654643],
       [ 1.2949088],
       [ 2.4586848],
       [ 2.5452118],
       [ 2.7160561],
       [ 2.2779684],
       [ 2.5390363],
       [ 2.0657391],
       [ 2.8313523],
       [ 3.2071277],
       [ 2.11

## Visualizing the Global Role Equivalence On Two Dimensional Space

To visualize this data in a meaningful way, I have used the manifold learning algorithms for metric multidimensional scaling from scikit-learn module in Python. Multidimensional scaling allows a two dimensional representation of the data by respecting the original distances of the high-dimensional space. In this case for my graphs, countries that have a higher correlation of trade products (stronger ties) are represented closer to each other. Countries that have a lower correlation are farther apart from each other. This is a nice way of visually representing this data because for my work my hypothesis that countries that have higher similarities in their trade behavior are more likely to experience crises in similar time periods

In [ ]:
mds = manifold.MDS(n_components=2, dissimilarity="precomputed", random_state=2)
mds

Below is the coordinates for the countries calculated for 1962:

In [ ]:
results = mds.fit_transform(mat)
results

Using matplotlib I created the figure below. In the figure, I color coded the countries that experienced a revolutionary situation with black dots. Other countries have white dots. I expect black dots to be in general closer to each other. I also expect that in the following year, the countries that are close to black dots should be more likely to experience a revolutionary situation 

In [ ]:
fig = plt.figure(figsize=(28.0,16.5))
fig.suptitle('Revolutionary Situations in 1962', fontsize=14, fontweight='bold')


plt.subplots_adjust(bottom = 0.1)

plt.scatter(
    results[:, 0], results[:, 1], marker = 'o', c=rev
    )


for label, x, y, in zip(countries, results[:, 0], results[:, 1]):
    plt.annotate(
    label,
    xy = (x, y), xytext = (-20, 20),
    textcoords = 'offset points', ha = 'right', va='bottom',
    bbox = dict(boxstyle = 'round,pad=0.5', fc = 'yellow', alpha = 0.5),
    arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0'))

leg = plt.legend(('Revolutions', 'Stable'), loc = 'lower right', frameon=True)    
leg.get_frame().set_edgecolor('b')ii
plt.show()

Elsewhere I have written a program that has functions to automate the commands I have used above. With the loop below I have generated a plot for each year in my dataset

In [ ]:
year = 1962
while year<2010:
    numbers = figures.inputs('data/'+str(year)+'_rev.csv', 'data/'+str(year)+'_rev_vector.csv')
    coor = figures.mdscoordinates(numbers['matrix'])
    figures.mdsfigure(coor, year, numbers['behavior'], numbers['country names'])
    year+=1
#    plt.pause(0.5)
    plt.close('all')

Below is a plot that displays how countries move across the graph from 1962 to 2009:

In [ ]:
year = 1962
while year<2010:
    plt.figure(figsize=(28.0, 16.5))
    img=mpimg.imread('images/'+str(year)+'.png')
    imgplot = plt.imshow(img)
    year+=1
    plt.pause(0.5)
    plt.close('all')

## Panel Data Analysis
### Logistic Regression for Panel Data with Fixed Effects using Stata

Sociologists are mainly interested in finding variables that have a statistically significant effect on the outcomes. I have merged multiple datasets from World Bank, Correlates of War Project, Center For Systemic Peace among many others to create a comprehensive dataset on various variables for these 70 countries for the years between 1962-2009.

I have built negative binomial and logistic regression models for testing the effects of role equivalence, structural equivalence, and trade cohesion measures on various sovereign crises situations. Below is an output from one of the many models I have built

In [33]:
%%stata -o panel
use "/Volumes/iMac HD/git/insight-demo/data/Dataset-conflict.dta"
sum


    Variable |       Obs        Mean    Std. Dev.       Min        Max
-------------+--------------------------------------------------------
        year |      3249    1985.567    13.77182       1962       2009
 CNTScountry |         0
       CCode |      3249    389.2718    262.1387          2        920
      CNTSid |         0
 coupsuccess |      3246    .0206408    .1464706          0          2
-------------+--------------------------------------------------------
 coupattempt |      3246     .025878    .1718442          0          2
    coupplot |      3246    .0077018    .0908909          0          2
 coupalleged |      3246    .0113986    .1145477          0          2
         arc |      3246    .0061614    .0782646          0          1
      deaths |      3246    35.95102    2018.484          0     115000
-------------+--------------------------------------------------------
   couptotal |      3246    .0656192    .2963689          0          5
  totaltrade |      3217  

In [34]:
panel

,year,CNTScountry,CCode,CNTSid,coupsuccess,coupattempt,coupplot,coupalleged,arc,deaths,...,independence_year,currency_crises,inflation_crises,stock_market_crash,sovereign_debtcrises_domestic,sovereign_debt_crisis_external,banking_crises,crisis_tally,_merge1,_mergeRRdata
0,1967.0,Algeria,615.0,0030,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,matched (3),matched (3)
1,1968.0,Algeria,615.0,0030,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,matched (3),matched (3)
2,1969.0,Algeria,615.0,0030,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,matched (3),matched (3)
3,1970.0,Algeria,615.0,0030,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,matched (3),matched (3)
4,1971.0,Algeria,615.0,0030,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,matched (3),matched (3)
5,1972.0,Algeria,615.0,0030,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,matched (3),matched (3)
6,1973.0,Algeria,615.0,0030,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,matched (3),matched (3)
7,1974.0,Algeria,615.0,0030,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,matched (3),matched (3)
8,1975.0,Algeria,615.0,0030,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,matched (3),matched (3)
9,1976.0,Algeria,615.0,0030,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,matched (3),matched (3)


In [52]:
%%stata
use "/Volumes/iMac HD/git/insight-demo/data/Dataset-conflict.dta"
xtset ccode year
gen revolutions_dum=1 if revolutions>0
replace revolutions_dum=0 if revolutions==0
table revolutions_dum
xtlogit revolutions_dum l.roleeqrevolutions l.NYGDPMKTPKDZG l.inttot l.crisis_tally l.c.SPPOPGROW##l.c.SPPOPGROW l.year , fe or


       panel variable:  ccode (unbalanced)
        time variable:  year, 1962 to 2009, but with gaps
                delta:  1 unit
(2757 missing values generated)
(2757 real changes made)

----------------------
revolutio |
ns_dum    |      Freq.
----------+-----------
        0 |      2,757
        1 |        492
----------------------
note: multiple positive outcomes within groups encountered.
note: 21 groups (772 obs) dropped because of all positive or
      all negative outcomes.

Iteration 0:   log likelihood = -764.86191  
Iteration 1:   log likelihood = -762.25228  
Iteration 2:   log likelihood = -762.24251  
Iteration 3:   log likelihood = -762.24251  

Conditional fixed-effects logistic regression   Number of obs      =      2006
Group variable: ccode                           Number of groups   =        47

                                                Obs per group: min =        14
                                                               avg =      42.7
          